In [ ]:
!pip3 install torch
!pip3 install torchvision
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib

In [ ]:
!nvidia-smi

In [1]:
import numpy as np
import pandas as pd
import torch

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')
mnist.data.shape, mnist.target.shape
x_data = mnist.data
y_data = mnist.target.astype(int)

from sklearn.model_selection import train_test_split

device = 'cpu'
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.20, random_state=42)
X_train, X_test = torch.Tensor(X_train.values), torch.Tensor(X_test.values)
y_train, y_test = torch.Tensor(y_train.values), torch.Tensor(y_test.values)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


torch.Size([56000, 784])
torch.Size([14000, 784])
torch.Size([56000])
torch.Size([14000])


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )
  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits

In [3]:
lr = 0.001
epochs=2000
model = MLP().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [4]:
loss_save_arr=[]
for i in range(epochs):
  model.train()
  optimizer.zero_grad()
  output = model(X_train)
  loss = criterion(output, y_train.long())

  loss.backward()
  optimizer.step()
  loss_save_arr.append(loss.data)
  if(i%100==0):
    print("====")
    print("epoch ", i)
    print('loss ', loss.data)
    _, pred = torch.max(output.data, axis=1)
    print("train_accuracy {:0.3f}".format(float((pred == y_train).sum())/y_train.size(0)))

    model.eval()
    with torch.no_grad():
      output = model(X_test)
      _, pred = torch.max(output.data, axis=1)
      print("test_accuracy {:0.3f}".format(float((pred == y_test).sum())/y_test.size(0)))


====
epoch  0
loss  tensor(2.4096)
train_accuracy 0.090
test_accuracy 0.091
====
epoch  100
loss  tensor(1.6791)
train_accuracy 0.613
test_accuracy 0.620
====
epoch  200
loss  tensor(1.2827)
train_accuracy 0.753
test_accuracy 0.753
====
epoch  300
loss  tensor(1.0483)
train_accuracy 0.807
test_accuracy 0.810
====
epoch  400
loss  tensor(0.8946)
train_accuracy 0.837
test_accuracy 0.840
====
epoch  500
loss  tensor(0.7865)
train_accuracy 0.857
test_accuracy 0.857
====
epoch  600
loss  tensor(0.7065)
train_accuracy 0.868
test_accuracy 0.868
====
epoch  700
loss  tensor(0.6450)
train_accuracy 0.877
test_accuracy 0.877
====
epoch  800
loss  tensor(0.5962)
train_accuracy 0.883
test_accuracy 0.883
====
epoch  900
loss  tensor(0.5565)
train_accuracy 0.888
test_accuracy 0.888
====
epoch  1000
loss  tensor(0.5235)
train_accuracy 0.892
test_accuracy 0.892
====
epoch  1100
loss  tensor(0.4956)
train_accuracy 0.896
test_accuracy 0.895
====
epoch  1200
loss  tensor(0.4717)
train_accuracy 0.899
test_

In [ ]:
class LogisticRegression(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)
  def forward(self, x):
    outputs = torch.sigmoid(self.linear(x))
    return outputs


In [ ]:
import torch.nn as nn
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(output.shape)
print(m.weight.shape)

In [ ]:
epochs = 1000
input_dim =784
output_dim = 10
lr = 0.01
model = LogisticRegression(input_dim, output_dim)
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr =lr)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(range(epochs), [e.to("cpu") for e in loss_save_arr])